Модуль для анализа пролонгации проектов и создания отчетов.

Этот модуль предоставляет функциональность для:
- Загрузки и обработки данных о пролонгации проектов
- Расчета коэффициентов пролонгации на 1 и 2 месяца
- Создания визуализаций и Excel-отчета с результатами анализа

In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.formatting.rule import ColorScaleRule
from google.colab import files
import io

Загружает и объединяет данные о пролонгации и финансовых показателях.
    
    Алгоритм обработки:
    1. Загрузка данных:
       - Чтение CSV файлов с данными о пролонгации и финансовых показателях
       - Проверка наличия необходимых колонок (id, month, AM)
    
    2. Объединение данных:
       - Слияние таблиц по полю 'id' с использованием left join
       - Сохранение всех записей из таблицы пролонгаций
    
    3. Определение месячных колонок:
       - Поиск колонок, содержащих годы 2022, 2023, 2024
       - Проверка наличия хотя бы одной месячной колонки
    
    Args:
        prolongations_path (str): Путь к файлу с данными о пролонгации
        financial_path (str): Путь к файлу с финансовыми данными
        
    Returns:
        tuple: (pd.DataFrame, list) - объединенные данные и список месячных колонок
        

In [ ]:
def load_and_process_data(prolongations_path, financial_path):
    try:
        prolongations = pd.read_csv(prolongations_path, encoding='utf-8')
        financial_data = pd.read_csv(financial_path, encoding='utf-8')

        print("\nИнформация о данных:")
        print(f"Количество строк в prolongations.csv: {len(prolongations)}")
        print(f"Количество строк в financial_data.csv: {len(financial_data)}")
        print("\nУникальные значения в колонке 'month':")
        print(prolongations['month'].unique())

        # Проверка необходимых столбцов
        if not all(col in prolongations.columns for col in ['id', 'month', 'AM']):
            print("Ошибка: В prolongations.csv отсутствуют необходимые столбцы")
            return None, None

        # Объединение данных
        data = pd.merge(prolongations, financial_data, on='id', how='left')
        print(f"\nКоличество строк после объединения: {len(data)}")

        # Определение месячных колонок
        month_cols = [col for col in data.columns if any(year in col for year in ['2022', '2023', '2024'])]
        if not month_cols:
            print("❌ Не найдены колонки с месяцами!")
            return None, None

        print("\nНайденные месячные колонки:")
        print(month_cols)

        return data, month_cols
    except Exception as e:
        print(f"Ошибка при загрузке данных: {str(e)}")
        return None, None

Рассчитывает коэффициенты пролонгации для каждого менеджера по месяцам.
    
    Алгоритм расчета:
    1. Коэффициент пролонгации на 1 месяц (1m):
       - Находит проекты, завершившиеся в предыдущем месяце
       - Исключает проекты со статусом 'стоп' или 'end'
       - Для каждого менеджера:
         * Суммирует отгрузки за последний месяц реализации проектов
         * Суммирует отгрузки за текущий месяц
         * Рассчитывает коэффициент как отношение: текущая_отгрузка / последняя_отгрузка
    
    2. Коэффициент пролонгации на 2 месяца (2m):
       - Находит проекты, завершившиеся два месяца назад
       - Исключает проекты со статусом 'стоп' или 'end'
       - Для каждого менеджера:
         * Отбирает проекты, которые не были пролонгированы в первый месяц
         * Суммирует отгрузки за последний месяц реализации этих проектов
         * Суммирует отгрузки за текущий месяц
         * Рассчитывает коэффициент как отношение: текущая_отгрузка / последняя_отгрузка
    
    Особенности:
    - Все числовые значения проходят через функцию safe_float_convert для корректной обработки
    - Если знаменатель равен 0, коэффициент принимается равным 0
    - Результаты округляются до 2 знаков после запятой
    
    Args:
        data (pd.DataFrame): Объединенные данные
        month_cols (list): Список колонок с месяцами
        
    Returns:
        pd.DataFrame: Результаты расчета коэффициентов со следующими колонками:
            - Месяц: текущий месяц анализа
            - Менеджер: имя менеджера
            - Коэф. пролонгации (1m): коэффициент на 1 месяц
            - Коэф. пролонгации (2m): коэффициент на 2 месяца
            - Отгрузка последнего месяца: сумма отгрузок в последний месяц
            - Отгрузка текущего месяца: сумма отгрузок в текущий месяц
            

In [ ]:
def calculate_prolongation_coefficients(data, month_cols):

    results = []

    print(f"\nНачинаем расчет коэффициентов. Всего месяцев: {len(month_cols)}")
    print(f"Месяцы для анализа: {month_cols}")

    # Для каждого месяца (кроме первого)
    for i in range(1, len(month_cols)):
        current_month = month_cols[i]
        prev_month = month_cols[i-1]
        prev_prev_month = month_cols[i-2] if i > 1 else None

        print(f"\nАнализ месяца {current_month}")
        print(f"Предыдущий месяц: {prev_month}")
        print(f"Пред-предыдущий месяц: {prev_prev_month}")

        # Получаем проекты, завершившиеся в предыдущем месяце
        completed_projects = data[
            (data['month'].str.strip().str.lower() == prev_month.lower()) |
            (data['month'].str.strip().str.lower().str.contains(prev_month.lower()))
        ].copy()

        print(f"Найдено проектов, завершившихся в {prev_month}: {len(completed_projects)}")
        if not completed_projects.empty:
            print("Примеры значений в колонке 'month':")
            print(completed_projects['month'].head())

        if completed_projects.empty:
            print(f"Пропускаем {current_month} - нет завершенных проектов в {prev_month}")
            continue

        # Исключаем проекты со 'стоп' или 'end'
        completed_projects = completed_projects[
            ~completed_projects[current_month].str.lower().isin(['стоп', 'end'])
        ]
        print(f"Проектов после исключения 'стоп' и 'end': {len(completed_projects)}")

        # Группируем по менеджерам
        managers = completed_projects['AM'].unique()
        print(f"Найдено менеджеров: {len(managers)}")

        for manager in managers:
            print(f"\nОбработка менеджера: {manager}")
            manager_projects = completed_projects[completed_projects['AM'] == manager]
            print(f"Проектов у менеджера: {len(manager_projects)}")

            try:
                def safe_float_convert(x):
                    """Преобразует строковое значение в число с учетом специальных случаев."""
                    if pd.isna(x) or str(x).strip() == '':
                        return 0.0
                    x_str = str(x).replace('\xa0', '').replace(' ', '')
                    if x_str.lower() == 'в ноль':
                        return 0.0
                    try:
                        # Извлекаем только цифры, точки и минусы
                        clean_num = ''.join(c for c in x_str if c.isdigit() or c in '.-')
                        return float(clean_num) if clean_num else 0.0
                    except:
                        return 0.0

                # Расчет коэффициента пролонгации на 1 месяц
                last_month_shipment = manager_projects[prev_month].apply(safe_float_convert).sum()
                current_month_shipment = manager_projects[current_month].apply(safe_float_convert).sum()

                print(f"Отгрузка за {prev_month}: {last_month_shipment}")
                print(f"Отгрузка за {current_month}: {current_month_shipment}")

                coef_1m = current_month_shipment / last_month_shipment if last_month_shipment > 0 else 0
                print(f"Коэффициент пролонгации 1m: {coef_1m}")

                # Расчет коэффициента пролонгации на 2 месяца
                coef_2m = 0
                if prev_prev_month:
                    # Проекты, завершившиеся два месяца назад
                    prev_completed = data[
                        (data['month'].str.strip().str.lower() == prev_prev_month.lower()) |
                        (data['month'].str.strip().str.lower().str.contains(prev_prev_month.lower()))
                    ].copy()

                    prev_completed = prev_completed[prev_completed['AM'] == manager]
                    print(f"Проектов, завершившихся в {prev_prev_month}: {len(prev_completed)}")

                    # Исключаем проекты со 'стоп' или 'end'
                    prev_completed = prev_completed[
                        ~prev_completed[current_month].str.lower().isin(['стоп', 'end'])
                    ]

                    # Находим проекты, которые не были пролонгированы в первый месяц
                    no_first_month_prolongation = prev_completed[
                        prev_completed[prev_month].apply(
                            lambda x: safe_float_convert(x) == 0
                        )
                    ]

                    print(f"Проектов без пролонгации в первый месяц: {len(no_first_month_prolongation)}")

                    if not no_first_month_prolongation.empty:
                        # Сумма отгрузки в последний месяц реализации
                        last_month_shipment_2m = no_first_month_prolongation[prev_prev_month].apply(safe_float_convert).sum()

                        # Сумма отгрузки в текущий месяц
                        current_month_shipment_2m = no_first_month_prolongation[current_month].apply(safe_float_convert).sum()

                        print(f"Отгрузка за {prev_prev_month} (для 2m): {last_month_shipment_2m}")
                        print(f"Отгрузка за {current_month} (для 2m): {current_month_shipment_2m}")

                        coef_2m = current_month_shipment_2m / last_month_shipment_2m if last_month_shipment_2m > 0 else 0
                        print(f"Коэффициент пролонгации 2m: {coef_2m}")

                results.append({
                    'Месяц': current_month,
                    'Менеджер': manager,
                    'Коэф. пролонгации (1m)': round(coef_1m, 2),
                    'Коэф. пролонгации (2m)': round(coef_2m, 2),
                    'Отгрузка последнего месяца': last_month_shipment,
                    'Отгрузка текущего месяца': current_month_shipment
                })

            except Exception as e:
                print(f"Ошибка при обработке менеджера {manager}: {str(e)}")
                continue

    if not results:
        print("Не удалось рассчитать ни одного коэффициента")
        return pd.DataFrame()

    return pd.DataFrame(results)

Создает визуализации и добавляет их в Excel-файл.
    
    Args:
        data (pd.DataFrame): Данные для визуализации
        writer (pd.ExcelWriter): Объект для записи в Excel
        
    Returns:
        bool: True если визуализации успешно созданы, False в случае ошибки

In [ ]:
def create_visualizations(data, writer):
    try:
        print("Начинаем создание визуализаций...")

        # Настройка стиля графиков
        plt.style.use('default')
        plt.rcParams.update({
            'figure.facecolor': 'white',
            'axes.facecolor': 'white',
            'axes.grid': True,
            'grid.alpha': 0.3,
            'axes.labelsize': 12,
            'axes.titlesize': 14,
            'xtick.labelsize': 10,
            'ytick.labelsize': 10
        })

        # Создаем фигуру с несколькими графиками
        fig = plt.figure(figsize=(20, 15))

        # 1. График средних коэффициентов пролонгации по месяцам
        plt.subplot(2, 2, 1)
        monthly_avg = data.groupby('Месяц')[['Коэф. пролонгации (1m)', 'Коэф. пролонгации (2m)']].mean()
        monthly_avg.plot(kind='bar', ax=plt.gca())
        plt.title('Средние коэффициенты пролонгации по месяцам')
        plt.xlabel('Месяц')
        plt.ylabel('Коэффициент')
        plt.xticks(rotation=45)
        plt.legend(['1 месяц', '2 месяца'])
        plt.grid(True, alpha=0.3)

        # 2. График отгрузок по месяцам
        plt.subplot(2, 2, 2)
        monthly_shipments = data.groupby('Месяц')[['Отгрузка последнего месяца', 'Отгрузка текущего месяца']].sum()
        monthly_shipments.plot(kind='bar', ax=plt.gca())
        plt.title('Сумма отгрузок по месяцам')
        plt.xlabel('Месяц')
        plt.ylabel('Сумма отгрузки')
        plt.xticks(rotation=45)
        plt.legend(['Последний месяц', 'Текущий месяц'])
        plt.grid(True, alpha=0.3)

        # 3. Тепловая карта коэффициентов по менеджерам и месяцам
        plt.subplot(2, 2, 3)
        pivot_1m = data.pivot_table(
            values='Коэф. пролонгации (1m)',
            index='Менеджер',
            columns='Месяц',
            aggfunc='mean'
        )
        sns.heatmap(pivot_1m, annot=True, cmap='RdYlGn', center=1, fmt='.2f')
        plt.title('Коэффициенты пролонгации (1m) по менеджерам')
        plt.xticks(rotation=45)

        # 4. Тепловая карта коэффициентов 2m
        plt.subplot(2, 2, 4)
        pivot_2m = data.pivot_table(
            values='Коэф. пролонгации (2m)',
            index='Менеджер',
            columns='Месяц',
            aggfunc='mean'
        )
        sns.heatmap(pivot_2m, annot=True, cmap='RdYlGn', center=1, fmt='.2f')
        plt.title('Коэффициенты пролонгации (2m) по менеджерам')
        plt.xticks(rotation=45)

        # Настройка общего вида
        plt.tight_layout()

        # Сохраняем график в BytesIO
        img_data = io.BytesIO()
        plt.savefig(img_data, format='png', dpi=300, bbox_inches='tight')
        img_data.seek(0)

        print("Графики созданы, сохраняем в Excel...")

        # Создаем лист для графиков
        worksheet = writer.book.create_sheet('Графики')
        print("Лист 'Графики' создан")

        # Добавляем изображение напрямую из BytesIO
        from openpyxl.drawing.image import Image
        img = Image(img_data)
        worksheet.add_image(img, 'A1')
        print("Изображение добавлено на лист")

        # Настраиваем размеры ячеек
        worksheet.column_dimensions['A'].width = 100
        worksheet.row_dimensions[1].height = 75
        print("Размеры ячеек настроены")

        plt.close()
        return True

    except Exception as e:
        print(f"Ошибка при создании визуализаций: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return False

Генерирует и сохраняет отчет в Excel-файл.
    
    Args:
        data (pd.DataFrame): Данные для отчета
        output_file (str): Путь к файлу для сохранения отчета
        
    Returns:
        bool: True если отчет успешно создан, False в случае ошибки

In [ ]:
def generate_and_save_report(data, output_file):
    if data is None or data.empty:
        print("Нет данных для отчета")
        return False

    try:
        print("Начинаем создание отчета...")

        # Создаем Excel writer
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            print("Excel writer создан")

            # Создаем визуализации до сохранения данных
            print("Создаем визуализации...")
            create_visualizations(data, writer)

            # Сохраняем основной отчет
            print("Сохраняем основной отчет...")
            data.to_excel(writer, sheet_name='Отчет по месяцам', index=False)

            # Создаем сводный отчет по менеджерам
            print("Создаем сводный отчет...")
            manager_summary = data.groupby('Менеджер').agg({
                'Коэф. пролонгации (1m)': 'mean',
                'Коэф. пролонгации (2m)': 'mean',
                'Отгрузка последнего месяца': 'sum',
                'Отгрузка текущего месяца': 'sum'
            }).round(2)

            # Переименовываем столбцы
            manager_summary = manager_summary.rename(columns={
                'Отгрузка последнего месяца': 'Общая сумма отгрузок завершенных проектов',
                'Отгрузка текущего месяца': 'Общая сумма отгрузок пролонгированных проектов'
            })

            # Добавляем итоговую строку в сводный отчет
            total_row = pd.DataFrame({
                'Менеджер': ['ИТОГО'],
                'Коэф. пролонгации (1m)': [manager_summary['Коэф. пролонгации (1m)'].mean()],
                'Коэф. пролонгации (2m)': [manager_summary['Коэф. пролонгации (2m)'].mean()],
                'Общая сумма отгрузок завершенных проектов': [manager_summary['Общая сумма отгрузок завершенных проектов'].sum()],
                'Общая сумма отгрузок пролонгированных проектов': [manager_summary['Общая сумма отгрузок пролонгированных проектов'].sum()]
            })

            # Объединяем сводный отчет с итоговой строкой
            manager_summary = pd.concat([manager_summary.reset_index(), total_row], ignore_index=True)
            manager_summary.to_excel(writer, sheet_name='Сводный отчет', index=False)

            # Форматирование
            print("Применяем форматирование...")
            for sheet_name in writer.sheets:
                worksheet = writer.sheets[sheet_name]

                # Стили для заголовков
                header_font = Font(bold=True, size=11, color="FFFFFF")
                header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
                header_alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

                # Стили для данных
                data_font = Font(size=10)
                data_alignment = Alignment(horizontal='right', vertical='center')
                data_alignment_text = Alignment(horizontal='left', vertical='center')

                # Стили для границ
                thin_border = Border(
                    left=Side(style='thin'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')
                )

                # Форматирование заголовков
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = header_alignment
                    cell.border = thin_border

                # Форматирование данных
                for row in worksheet.iter_rows(min_row=2):
                    for cell in row:
                        cell.border = thin_border
                        cell.font = data_font

                        # Форматирование в зависимости от типа данных
                        if cell.column in [1, 2]:  # Месяц/Менеджер
                            cell.alignment = data_alignment_text
                        else:  # Числовые данные
                            cell.alignment = data_alignment
                            if cell.column in [3, 4]:  # Коэффициенты
                                cell.number_format = '0.00'
                            else:  # Отгрузки
                                cell.number_format = '#,##0'

                # Автоподбор ширины столбцов
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter

                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass

                    adjusted_width = (max_length + 2) * 1.2
                    worksheet.column_dimensions[column_letter].width = adjusted_width

                # Замораживаем заголовки
                worksheet.freeze_panes = 'A2'

                # Добавляем условное форматирование для коэффициентов
                if sheet_name == 'Отчет по месяцам':
                    # Цветовая шкала для коэффициентов
                    worksheet.conditional_formatting.add(
                        f'C2:C{len(data)+1}',
                        ColorScaleRule(
                            start_type='num', start_value=0, start_color='FF0000',
                            mid_type='num', mid_value=1, mid_color='FFFFFF',
                            end_type='num', end_value=2, end_color='00FF00'
                        )
                    )
                    worksheet.conditional_formatting.add(
                        f'D2:D{len(data)+1}',
                        ColorScaleRule(
                            start_type='num', start_value=0, start_color='FF0000',
                            mid_type='num', mid_value=1, mid_color='FFFFFF',
                            end_type='num', end_value=2, end_color='00FF00'
                        )
                    )

                # Форматирование итоговой строки в сводном отчете
                if sheet_name == 'Сводный отчет':
                    last_row = len(manager_summary) + 1
                    for cell in worksheet[last_row]:
                        cell.font = Font(bold=True)
                        cell.border = thin_border
                        if cell.column > 1:  # Для числовых столбцов
                            cell.alignment = data_alignment
                            cell.number_format = '#,##0' if cell.column > 4 else '0.00'

        print(f"Отчет сохранен как {output_file}")
        files.download(output_file)
        return True

    except Exception as e:
        print(f"Ошибка при сохранении отчета: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return False

Вызов программы

In [ ]:
if __name__ == "__main__":
    prolongations_file = 'prolongations.csv'
    financial_file = 'financial_data.csv'
    output_file = 'отчет_пролонгации.xlsx'

    # Загрузка и обработка данных
    data, month_cols = load_and_process_data(prolongations_file, financial_file)

    if data is not None and month_cols is not None:
        # Расчет коэффициентов
        results = calculate_prolongation_coefficients(data, month_cols)

        if not results.empty:
            print("\nРезультаты расчета:")
            print(results.head())

            # Генерация и сохранение отчета
            generate_and_save_report(results, output_file)
        else:
            print("Не удалось рассчитать коэффициенты")
    else:
        print("Не удалось загрузить данные")


Информация о данных:
Количество строк в prolongations.csv: 477
Количество строк в financial_data.csv: 451

Уникальные значения в колонке 'month':
['ноябрь 2022' 'декабрь 2022' 'январь 2023' 'февраль 2023' 'март 2023'
 'апрель 2023' 'май 2023' 'июнь 2023' 'июль 2023' 'август 2023'
 'сентябрь 2023' 'октябрь 2023' 'ноябрь 2023' 'декабрь 2023']

Количество строк после объединения: 725

Найденные месячные колонки:
['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']

Начинаем расчет коэффициентов. Всего месяцев: 16
Месяцы для анализа: ['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']

Анализ месяца Декабрь 2022
Предыдущий

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>